In [1]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import json
import gc
from sklearn.kernel_ridge import KernelRidge
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import spacy
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.decomposition import TruncatedSVD
from transformers import AutoTokenizer, AutoModel ,BertForTokenClassification,TFBertForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer
#from thundersvm import SVR
from sklearn.model_selection import RandomizedSearchCV
# path_to_json = 'judgments_ECtHR/judgments_ECtHR/'
# json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
spacy_nlp = spacy.load('C:\\Users\\sichi\\PycharmProjects\\NLPproiect\\venv\\Lib\\site-packages\\en_core_web_sm\\en_core_web_sm-2.3.1')
months = ["january","february","march","april","may","june","july","august","september","october","november","december"]

df = pd.DataFrame()
df = pd.read_csv('input.csv', dtype={col:str for col in df.columns})
years = [str(i) for i in range(1960,2021)]

with open("ners2.txt", "rb") as fp:
    ners = set(pickle.load(fp))

def clean_numbers(row,col):
    #year = row['kpdate'].split('-')[0]
    article = row['appno']
    if article is None:
        article = row['extractedappno']
    for year in years:
        row[col] = row[col].replace(year,'YEAR')
    for x in article.split(';'):
        row[col] = row[col].replace(x,'APPNUMBER')
    if(re.search("^\s*[0-9]+\.",str(row['law']))):
        app_n = str(row['law']).split('.')[0] + '.'
        row[col] = row[col].replace(app_n,'NUMBER')
    return row[col]

df['law'] = df['law'].apply(lambda x: str(x).lower())
for index, row in df.iterrows():
    df.at[index, 'law'] = clean_numbers(row,'law')
df['law'] = df['law'].apply(lambda x: spacy_nlp.tokenizer(x))
df['law'] = df['law'].apply(lambda sentence: ' '.join([token.lemma_ for token in sentence if token.is_stop == False and token.is_punct == False and token.text not in ners and token.text not in years]))

df['kpdate'] = df['kpdate'].apply(lambda date: float(int(date.split('-')[0]) + float((int(date.split('-')[1]) - 1)/12)))

In [14]:
sorted([(len(x),i) for i,x in enumerate(df['law'])],key=lambda tup:tup[0],reverse=True)

[(557677, 5891),
 (551705, 5890),
 (398640, 3013),
 (398103, 3012),
 (298355, 2317),
 (288551, 4855),
 (259523, 4696),
 (219577, 3730),
 (190309, 9806),
 (184750, 9708),
 (174070, 7118),
 (162000, 4970),
 (156479, 8776),
 (152210, 5368),
 (140200, 1943),
 (135353, 827),
 (134780, 3197),
 (132801, 6968),
 (132590, 3497),
 (132259, 6047),
 (131403, 2182),
 (128912, 6279),
 (127327, 3546),
 (125830, 1179),
 (122976, 4606),
 (121646, 7667),
 (120084, 2908),
 (111475, 10156),
 (105934, 14837),
 (100187, 5508),
 (99557, 2892),
 (98918, 3046),
 (95088, 2966),
 (94319, 10749),
 (93870, 498),
 (93531, 3059),
 (92507, 6715),
 (91445, 14718),
 (91388, 2592),
 (90940, 13087),
 (90524, 923),
 (90170, 6230),
 (89776, 877),
 (89099, 3834),
 (87259, 5559),
 (87143, 643),
 (85931, 7147),
 (85244, 6146),
 (84340, 7002),
 (82653, 13373),
 (82596, 362),
 (82052, 13700),
 (81071, 3288),
 (80989, 6209),
 (80855, 2893),
 (80847, 1944),
 (80766, 5680),
 (80460, 9309),
 (80343, 758),
 (80152, 3597),
 (79997, 5

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df['law'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,2),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [5]:
#46 1.6812650223993555
#for c in range(1,100,1):
svr = LinearSVR(C=46) #1.6812650223993555
svr.fit(X_train,y_train)
predicted = svr.predict(X_test)

mean = mean_absolute_error(y_test, predicted)
print(str(c),str(mean))

1 3.2849793143847417
2 2.2273391292193874
3 1.9499670172670376
4 1.8419801746615188
5 1.784331665780273
6 1.7506755311981115
7 1.7298789540782662
8 1.7186790375708743
9 1.7072502133023584
10 1.7015082583516405
11 1.697549941850898
12 1.6940783494920884
13 1.6920462774845562
14 1.690627818900557
15 1.689781967052603
16 1.6888137758012354
17 1.687955737934993
18 1.6871277969128018
19 1.6868787799898635
20 1.6858124402390036
21 1.686148578408835
22 1.6859975422622175
23 1.686212758462715
24 1.6858505699082085
25 1.6881229661017525
26 1.6856309422239666
27 1.6851961710231629
28 1.6850720708833253
29 1.6849250230331854
30 1.684953846851653
31 1.6845709683475403
32 1.6844126330818552
33 1.6841659084410041
34 1.6839582196633993


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


35 1.6864548024390313


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


36 1.683284704555259


...

c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


45 1.6814129260903627


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


46 1.6812650223993555


KeyboardInterrupt: 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['law'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,1),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [7]:
svr = LinearSVR(C=11) #1.7661893668756792
svr.fit(X_train,y_train)
predicted = svr.predict(X_test)

mean = mean_absolute_error(y_test, predicted)
print(str(c),str(mean))

1 7.688321737862106
2 3.1924735511032556
3 2.3254067794728046
4 2.042787009616559
5 1.9093648340807936
6 1.8447505134166267
7 1.8096217540324033
8 1.7893991519603882
9 1.7765162872855051
10 1.768727825366124
11 1.7661893668756792
12 1.767613065691249
13 1.7705416465758146
14 1.7734433233811924
15 1.7754550740097335
16 1.7783664527256742
17 1.781325267674323
18 1.784665843684034
19 1.78924899178277
20 1.7934315549300188
21 1.7989112157625777
22 1.8045104579424505
23 1.8103295293207424
24 1.8160890392292248
25 1.8222564387294815
26 1.8280632925341889
27 1.834222916729126
28 1.8399895996849325


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


29 1.845264529313877


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


30 1.8505308334331059


...

c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


47 1.9224452775387577


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


48 1.9250294078088817
49 1.9282360430460561


c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['law'], df['kpdate'], test_size=0.1, random_state=0)
tf_vectorizer=TfidfVectorizer(ngram_range=(1,2),encoding='utf-8',sublinear_tf=True) #min_df=0 , #binary maybe sublinear_tf=true
X_train=tf_vectorizer.fit_transform(X_train)
X_test=tf_vectorizer.transform(X_test)

In [16]:
clf = KernelRidge(kernel='rbf',alpha=0.001,gamma = 0.02) #0.02
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
mean = mean_absolute_error(y_test, predicted)
print(mean) #1.3149833926036976

1.3902686776352113
1.3277040438140446
1.3194492666307926
1.3167178913177084
1.3155312316677505
1.3149833926036976
1.3148902019108486


KeyboardInterrupt: 

In [5]:
feature_names = tf_vectorizer.get_feature_names()
coefs_with_fns = sorted(zip([abs(x) for x in svr.coef_], feature_names))
df_coefs=pd.DataFrame(coefs_with_fns)
df_coefs.columns='coefficient','word'
df_coefs = df_coefs.sort_values(by='coefficient',ascending=False)

MemoryError: Unable to allocate 10.6 GiB for an array with shape (1088440, 1310) and data type float64

In [ ]:
#for c in np.linspace(20,100,5):
#for gam in np.linspace(20,30,5):
clf = SVR(kernel='rbf',C=20,gamma = 2.75) #1.1334412177708895
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
mean = mean_absolute_error(y_test, predicted)
print(str(c),str(gam),str(mean))

100.0 20.0 1.1334412177708895
100.0 22.5 1.1336548344020452
100.0 25.0 1.1338480945153688
